In [12]:
# ============================================================================
# STEP 1: INSTALL REQUIRED PACKAGES (WINDOWS SAFE)
# ============================================================================
print("="*70)
print(" INSTALLING PACKAGES")
print("="*70)
print("This will take 2-3 minutes. Please wait...")
print("(Dependency warnings are normal and can be ignored)\n")

!pip install -q langchain langchain-community langchain-google-genai sentence-transformers chromadb google-generativeai google-search-results pandas

print("\n Installation complete!")
print("="*70)


 INSTALLING PACKAGES
This will take 2-3 minutes. Please wait...
(Dependency warnings are normal and can be ignored)


 Installation complete!


In [3]:
pip install "protobuf<=4.21.12,>=4.21.1" --force-reinstall


Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-4.21.12-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.21.12-cp310-abi3-win_amd64.whl (527 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.6
    Uninstalling protobuf-5.29.6:
      Successfully uninstalled protobuf-5.29.6
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.21.12 which is incompatible.
opentelemetry-proto 1.39.1 requires protobuf<7.0,>=5.0, but you have protobuf 4.21.12 which is incompatible.


In [13]:
# ============================================================================
# STEP 2: INSTALL REQUIRED PACKAGES (UPDATED)
# ============================================================================
print("="*70)
print(" INSTALLING REQUIRED PACKAGES")
print("="*70)

!pip install -q \
langchain \
langchain-community \
langchain-core \
langchain-text-splitters \
langchain-google-genai \
chromadb \
sentence-transformers \
google-generativeai \
google-search-results \
pandas

print("\n Installation complete!")
print("="*70)


 INSTALLING REQUIRED PACKAGES

 Installation complete!


In [14]:
# Step: Set up Gemini API Key (Jupyter / Local)
import os

api_key = os.getenv("GEMINI_API_KEY") or os.getenv("GEM_API_KEY")

if not api_key:
    raise RuntimeError(
        "Gemini API key not found. "
        "Set it as an environment variable: GEMINI_API_KEY"
    )

os.environ["GOOGLE_API_KEY"] = api_key
print("Gemini API key configured.")


Gemini API key configured.


In [15]:
# Standard Python
import os
import getpass
import pandas as pd
from datetime import datetime

# LangChain – NEW modular structure
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [16]:
print("\nStep: Creating delivery service knowledge base...")

documents = [
    Document(
        page_content="The delivery service provides doorstep pickup and drop services for parcels, groceries, food items, and essential goods within designated service areas.",
        metadata={"source": "delivery_overview"}
    ),
    Document(
        page_content="Delivery services operate across local, intra-city, and inter-city routes depending on customer requirements and service availability.",
        metadata={"source": "delivery_coverage"}
    ),
    Document(
        page_content="Customers can place delivery requests through a mobile application, website, or customer support helpline. Each order is assigned a unique tracking ID.",
        metadata={"source": "order_placement"}
    ),
    Document(
        page_content="Standard delivery options include same-day delivery, next-day delivery, and scheduled delivery based on distance, package size, and urgency.",
        metadata={"source": "delivery_types"}
    ),
    Document(
        page_content="Express delivery provides faster service with higher priority handling and reduced delivery time compared to standard delivery.",
        metadata={"source": "express_delivery"}
    ),
    Document(
        page_content="Delivery charges are calculated based on distance, package weight, package dimensions, and delivery speed selected by the customer.",
        metadata={"source": "pricing_policy"}
    ),
    Document(
        page_content="Real-time order tracking allows customers to monitor the delivery status, estimated arrival time, and courier location using the tracking ID.",
        metadata={"source": "tracking_system"}
    ),
    Document(
        page_content="The delivery service follows strict safety and handling guidelines to ensure packages are transported securely and delivered without damage.",
        metadata={"source": "safety_guidelines"}
    ),
    Document(
        page_content="In case of delayed, lost, or damaged packages, customers can raise a support request through the app or customer service for resolution.",
        metadata={"source": "issue_resolution"}
    ),
    Document(
        page_content="Delivery personnel are verified and trained to follow professional conduct, customer privacy standards, and contactless delivery protocols where applicable.",
        metadata={"source": "delivery_personnel"}
    ),
]



Step: Creating delivery service knowledge base...


In [17]:
#step:5 chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

splits = text_splitter.split_documents(documents)
print(f" Created {len(splits)} document chunks")



 Created 10 document chunks


In [18]:
#step:6 embeddings
print("\n loading")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("embedded model loaded")


 loading


C:\Users\David\AppData\Local\Temp\ipykernel_23712\681121309.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


embedded model loaded


In [19]:
#step 7 : Create vector store
print("\n creating vector db")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="crd"
)
print("created")


 creating vector db
created


In [20]:
#step 8 retirever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

In [21]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage


In [22]:
print("\n Initializing Gemini...")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_output_tokens=1024,
)

print(" Gemini ready")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



 Initializing Gemini...
 Gemini ready


In [23]:
# Conversation buffer memory (NEW LangChain)
chat_history = []
print("Conversation memory initialized")

print("converse memory")

Conversation memory initialized
converse memory


In [24]:
from langchain_core.prompts import PromptTemplate

QA_PROMPT = PromptTemplate.from_template(
    """You are a helpful assistant answering questions using retrieved documents.

Chat History:
{chat_history}

Context:
{context}

Question:
{question}

Answer:"""
)


In [25]:
print("building conversational RAG chain")

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

conversational_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "chat_history": RunnablePassthrough()
    }
    | QA_PROMPT
    | llm
    | StrOutputParser()
)

print("conversational RAG chain ready!")


building conversational RAG chain
conversational RAG chain ready!


In [28]:
def ask(query):
    global chat_history

    print(f"\nUSER: {query}")

    # Retrieve documents explicitly (for showing sources only)
    docs = retriever.invoke(query)

    # Invoke LCEL chain with STRING input ONLY
    answer = conversational_chain.invoke(query)

    print(f"\nAI: {answer}")

    # Show sources
    print("\nSources used:")
    for i, doc in enumerate(docs, 1):
        print(f"  {i}. {doc.metadata.get('source', 'unknown')}")

    # Update buffer memory
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=answer))

    print("-" * 60)


In [29]:
ask("What delivery options are available?")
ask("How are delivery charges calculated?")
ask("Can customers track their orders?")
ask("What happens if a package is lost or damaged?")



USER: What delivery options are available?


ChatGoogleGenerativeAIError: Error calling model 'gemini-1.5-flash' (NOT_FOUND): 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}